<a href="https://colab.research.google.com/github/ssheggrud/Mod_20_Project/blob/05_Riley/2001_Contributions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ***2001 Indivdual Contributions***

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
#spark_version = 'spark-2.4.8'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

import pandas as pd


Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bioni

In [2]:
import findspark
findspark.add_packages('mysql:mysql-connector-java:8.0.11')

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-09-17 20:13:04--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  5.96MB/s    in 0.2s    

2021-09-17 20:13:05 (5.96 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [4]:
# Start Spark session
#from pyspark.sql import SparkSession
#from pyspark.sql.functions import col
#spark = SparkSession.builder.appName("FinalProject").getOrCreate()

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://databootcamp-final-05.s3.amazonaws.com/Resources/Individual+Contributions+2001.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("Individual+Contributions+2001.csv"), sep=",", header=True)
df.show()

+--------+--------+------+--------+-----------+---------+------+---------+------+--------+------+----------+--------+---------------+----------+----+------+-----+-------+---------+---------+----+-----+-----+-----------------+--------------------+--------+------------------+--------+--------+----+---------+--------+----------+--------+----------+--------+--------+--------+-------+-----+------+----------+----------+----------+----------+--------+---------+---------+--------+---------+------+-------+----------+
|ELECTION|OFFICECD|CANDID|CANCLASS|   CANDLAST|CANDFIRST|CANDMI|COMMITTEE|FILING|SCHEDULE|PAGENO|SEQUENCENO|   REFNO|           DATE|REFUNDDATE|NAME|C_CODE|STRNO|STRNAME|APARTMENT|BOROUGHCD|CITY|STATE|  ZIP|       OCCUPATION|             EMPNAME|EMPSTRNO|        EMPSTRNAME| EMPCITY|EMPSTATE|AMNT|MATCHAMNT|PREVAMNT|PAY_METHOD|INTERMNO|INTERMNAME|INTSTRNO|INTSTRNM|INTAPTNO|INTCITY|INTST|INTZIP|INTEMPNAME|INTEMPSTNO|INTEMPSTNM|INTEMPCITY|INTEMPST|INTOCCUPA|PURPOSECD|EXEMPTCD|ADJTYPECD

In [7]:
from pyspark.sql.functions import concat_ws
df2 = df.withColumn("CandidateName", concat_ws(",","CANDFIRST",'CANDLAST')) 
df2.show()

+--------+--------+------+--------+-----------+---------+------+---------+------+--------+------+----------+--------+---------------+----------+----+------+-----+-------+---------+---------+----+-----+-----+-----------------+--------------------+--------+------------------+--------+--------+----+---------+--------+----------+--------+----------+--------+--------+--------+-------+-----+------+----------+----------+----------+----------+--------+---------+---------+--------+---------+------+-------+----------+--------------------+
|ELECTION|OFFICECD|CANDID|CANCLASS|   CANDLAST|CANDFIRST|CANDMI|COMMITTEE|FILING|SCHEDULE|PAGENO|SEQUENCENO|   REFNO|           DATE|REFUNDDATE|NAME|C_CODE|STRNO|STRNAME|APARTMENT|BOROUGHCD|CITY|STATE|  ZIP|       OCCUPATION|             EMPNAME|EMPSTRNO|        EMPSTRNAME| EMPCITY|EMPSTATE|AMNT|MATCHAMNT|PREVAMNT|PAY_METHOD|INTERMNO|INTERMNAME|INTSTRNO|INTSTRNM|INTAPTNO|INTCITY|INTST|INTZIP|INTEMPNAME|INTEMPSTNO|INTEMPSTNM|INTEMPCITY|INTEMPST|INTOCCUPA|PURPOSE

In [8]:
# Remove multiple columns
list = ['COMMITTEE', 'CANDMI', 'CANDLAST', 'CANDFIRST','FILING', 'SCHEDULE', 'PAGENO', 'SEQUENCENO','REFNO', 'STRNO', 'STRNAME', 'APARTMENT', 'OCCUPATION', 'EMPNAME', 'EMPSTATE', 'EMPSTRNO', 'EMPSTRNAME',
                'EMPCITY','INTERMNO', 'REFUNDDATE', 'INTERMNAME', 'INTSTRNO', 'INTSTRNM', 'INTSTRNM', 'INTAPTNO',
                'INTCITY', 'INTST','INTZIP', 'INTEMPNAME', 'INTEMPSTNO', 'INTEMPSTNM', 'INTEMPCITY', 'INTEMPST', 'INTOCCUPA' ,'PURPOSECD', 'EXEMPTCD','ADJTYPECD', 'RR_IND', 'SEG_IND','INT_C_CODE'] 
df3 = df2.drop(*list)
df3.show()

+--------+--------+------+--------+---------------+----+------+---------+----+-----+-----+----+---------+--------+----------+--------------------+
|ELECTION|OFFICECD|CANDID|CANCLASS|           DATE|NAME|C_CODE|BOROUGHCD|CITY|STATE|  ZIP|AMNT|MATCHAMNT|PREVAMNT|PAY_METHOD|       CandidateName|
+--------+--------+------+--------+---------------+----+------+---------+----+-----+-----+----+---------+--------+----------+--------------------+
|    2001|       5|   579|       P|  8/8/2001 0:00|null|   IND|        M|  NY|   NY|10034|  10|       10|       0|         2|      Michael,Hughes|
|    2001|       5|   559|       P| 8/31/2001 0:00|null|  OTHR|     null|null| null| null|   1|        0|       0|         0|       Donald,Wooten|
|    2001|       5|    C6|       P|  8/1/2000 0:00|null|  CORP|     null|null| null| null| 100|        0|       0|         2|      Albert,Fabozzi|
|    2001|       2|   366|       P|  7/5/2001 0:00|null|  UNKN|     null|null| null| null|  25|        0|       0|    

In [9]:
#Rearrange columns
df4=df3.select('ELECTION','OFFICECD','CANDID','CANCLASS','CandidateName', 'DATE', 'NAME','C_CODE','BOROUGHCD','CITY','STATE', 'ZIP', 'AMNT', 'MATCHAMNT', 'PREVAMNT', 'PAY_METHOD')

In [10]:
# check data types
df4.dtypes

[('ELECTION', 'string'),
 ('OFFICECD', 'string'),
 ('CANDID', 'string'),
 ('CANCLASS', 'string'),
 ('CandidateName', 'string'),
 ('DATE', 'string'),
 ('NAME', 'string'),
 ('C_CODE', 'string'),
 ('BOROUGHCD', 'string'),
 ('CITY', 'string'),
 ('STATE', 'string'),
 ('ZIP', 'string'),
 ('AMNT', 'string'),
 ('MATCHAMNT', 'string'),
 ('PREVAMNT', 'string'),
 ('PAY_METHOD', 'string')]

In [11]:
#Change column names
df5 = df4 \
.withColumnRenamed("ELECTION", "Election") \
.withColumnRenamed("CANDID", "CandidateID") \
.withColumnRenamed("DATE", "Date") \
.withColumnRenamed("NAME", "Name") \
.withColumnRenamed("C_CODE", "ContributorType") \
.withColumnRenamed("BOROUGHCD", "BoroughName") \
.withColumnRenamed("CITY", "City") \
.withColumnRenamed("STATE", "State") \
.withColumnRenamed("ZIP", "ZipCode") \
.withColumnRenamed("EMPSTATE", "ContributionState") \
.withColumnRenamed("AMNT", "Amount") \
.withColumnRenamed("MATCHAMNT", "MatchAmt") \
.withColumnRenamed("PREVAMNT", "PrevAmt") \
.withColumnRenamed("PAY_METHOD", "PayMethod")

df5.printSchema()

root
 |-- Election: string (nullable = true)
 |-- OFFICECD: string (nullable = true)
 |-- CandidateID: string (nullable = true)
 |-- CANCLASS: string (nullable = true)
 |-- CandidateName: string (nullable = false)
 |-- Date: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- ContributorType: string (nullable = true)
 |-- BoroughName: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- ZipCode: string (nullable = true)
 |-- Amount: string (nullable = true)
 |-- MatchAmt: string (nullable = true)
 |-- PrevAmt: string (nullable = true)
 |-- PayMethod: string (nullable = true)



In [12]:
df5.show()

+--------+--------+-----------+--------+--------------------+---------------+----+---------------+-----------+----+-----+-------+------+--------+-------+---------+
|Election|OFFICECD|CandidateID|CANCLASS|       CandidateName|           Date|Name|ContributorType|BoroughName|City|State|ZipCode|Amount|MatchAmt|PrevAmt|PayMethod|
+--------+--------+-----------+--------+--------------------+---------------+----+---------------+-----------+----+-----+-------+------+--------+-------+---------+
|    2001|       5|        579|       P|      Michael,Hughes|  8/8/2001 0:00|null|            IND|          M|  NY|   NY|  10034|    10|      10|      0|        2|
|    2001|       5|        559|       P|       Donald,Wooten| 8/31/2001 0:00|null|           OTHR|       null|null| null|   null|     1|       0|      0|        0|
|    2001|       5|         C6|       P|      Albert,Fabozzi|  8/1/2000 0:00|null|           CORP|       null|null| null|   null|   100|       0|      0|        2|
|    2001|      

In [13]:
from datetime import datetime
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType

In [14]:
# using lambda function to convert date col to datetype
# changing datatypes of columns
#func =  udf (lambda x: datetime.strptime(x, '%m/%d/%Y'), DateType())
#df6 = df5.withColumn("Election", df5["Election"].cast("Integer")) \
 # .withColumn("OFFICECD", df5["OFFICECD"].cast("Integer")) \
  #.withColumn("ZipCode", df5["ZipCode"].cast("Integer")) \
  #.withColumn("Amount", df5["Amount"].cast("Float")) \
  #.withColumn("MatchAmt", df5["MatchAmt"].cast("Float")) \
  #.withColumn("PrevAmt", df5["PrevAmt"].cast("Float")) \
  #.withColumn('Date', func(col('Date')))

#df6.printSchema()

#Keep CandidateID as string
#.withColumn("CandidateID", df4["CandidateID"].cast("Integer")) \


In [15]:
df5.show()

+--------+--------+-----------+--------+--------------------+---------------+----+---------------+-----------+----+-----+-------+------+--------+-------+---------+
|Election|OFFICECD|CandidateID|CANCLASS|       CandidateName|           Date|Name|ContributorType|BoroughName|City|State|ZipCode|Amount|MatchAmt|PrevAmt|PayMethod|
+--------+--------+-----------+--------+--------------------+---------------+----+---------------+-----------+----+-----+-------+------+--------+-------+---------+
|    2001|       5|        579|       P|      Michael,Hughes|  8/8/2001 0:00|null|            IND|          M|  NY|   NY|  10034|    10|      10|      0|        2|
|    2001|       5|        559|       P|       Donald,Wooten| 8/31/2001 0:00|null|           OTHR|       null|null| null|   null|     1|       0|      0|        0|
|    2001|       5|         C6|       P|      Albert,Fabozzi|  8/1/2000 0:00|null|           CORP|       null|null| null|   null|   100|       0|      0|        2|
|    2001|      

In [16]:
#Change vaule name in ContributerType
from pyspark.sql.functions import regexp_replace
df7 = df5.withColumn('ContributorType', regexp_replace('ContributorType', 'CAN', 'Candidate')) \
  .withColumn('ContributorType', regexp_replace('ContributorType', 'CORP', 'Corporation')) \
  .withColumn('ContributorType', regexp_replace('ContributorType', 'EMPO', 'Labor Union')) \
  .withColumn('ContributorType', regexp_replace('ContributorType', 'FAM', 'Candidate Family')) \
  .withColumn('ContributorType', regexp_replace('ContributorType', 'IND', 'Individual')) \
  .withColumn('ContributorType', regexp_replace('ContributorType', 'PCOMZ', 'Party Committee')) \
  .withColumn('ContributorType', regexp_replace('ContributorType', 'ORG', 'Orgainization')) \
  .withColumn('ContributorType', regexp_replace('ContributorType', 'LLC', 'Limited Liability Company')) \
  .withColumn('ContributorType', regexp_replace('ContributorType', 'PCOMP', 'Political Action Committee')) \
  .withColumn('ContributorType', regexp_replace('ContributorType', 'SPO', 'Spouse')) \
  .withColumn('ContributorType', regexp_replace('ContributorType', 'OTHR', 'Other')) \
  .withColumn('ContributorType', regexp_replace('ContributorType', 'PCOMC', 'Candidate Committee')) \
  .withColumn('ContributorType', regexp_replace('ContributorType', 'PART', 'Individual')) \


In [17]:
df7.show()

+--------+--------+-----------+--------+--------------------+---------------+----+--------------------+-----------+----+-----+-------+------+--------+-------+---------+
|Election|OFFICECD|CandidateID|CANCLASS|       CandidateName|           Date|Name|     ContributorType|BoroughName|City|State|ZipCode|Amount|MatchAmt|PrevAmt|PayMethod|
+--------+--------+-----------+--------+--------------------+---------------+----+--------------------+-----------+----+-----+-------+------+--------+-------+---------+
|    2001|       5|        579|       P|      Michael,Hughes|  8/8/2001 0:00|null|          Individual|          M|  NY|   NY|  10034|    10|      10|      0|        2|
|    2001|       5|        559|       P|       Donald,Wooten| 8/31/2001 0:00|null|               Other|       null|null| null|   null|     1|       0|      0|        0|
|    2001|       5|         C6|       P|      Albert,Fabozzi|  8/1/2000 0:00|null|         Corporation|       null|null| null|   null|   100|       0|     

In [18]:
#Change Payment Method Name
df8=df7.withColumn('PayMethod', regexp_replace('PayMethod','0','Unknown')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','1','Cash')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','2','Check')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','3','Other')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','4','Credit Card')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','5','Money Order')) \

In [19]:
df8.show()

+--------+--------+-----------+--------+--------------------+---------------+----+--------------------+-----------+----+-----+-------+------+--------+-------+---------+
|Election|OFFICECD|CandidateID|CANCLASS|       CandidateName|           Date|Name|     ContributorType|BoroughName|City|State|ZipCode|Amount|MatchAmt|PrevAmt|PayMethod|
+--------+--------+-----------+--------+--------------------+---------------+----+--------------------+-----------+----+-----+-------+------+--------+-------+---------+
|    2001|       5|        579|       P|      Michael,Hughes|  8/8/2001 0:00|null|          Individual|          M|  NY|   NY|  10034|    10|      10|      0|    Check|
|    2001|       5|        559|       P|       Donald,Wooten| 8/31/2001 0:00|null|               Other|       null|null| null|   null|     1|       0|      0|  Unknown|
|    2001|       5|         C6|       P|      Albert,Fabozzi|  8/1/2000 0:00|null|         Corporation|       null|null| null|   null|   100|       0|     

In [20]:
#Change Borough name
df9 = df8.withColumn('BoroughName', regexp_replace('BoroughName', 'K', 'Brooklyn')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'M', 'Manhattan')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'Q', 'Queens')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'S', 'Staten Island')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'X', 'Bronx')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'Z', 'Other')) \

In [21]:
#Call only Mayor (1) and particaptes (P) within Dataframe
df10 = df9.filter((df8.OFFICECD=="1") & (df8.CANCLASS=="P"))

In [22]:
#Check the above dataframe to make sure only unique values between the OFFICECD is 1 
df10.select('OFFICECD').distinct().show()

+--------+
|OFFICECD|
+--------+
|       1|
+--------+



In [23]:
#Drop OFFICECD and CANCLASS
list2 = ['OFFICECD', 'CANCLASS'] 
df11 = df10.drop(*list2)

In [24]:
#Filter Dataframe removing the null CandidateID values
df12 = df11.filter(df10.CandidateID.isNotNull())

In [25]:
# ADD AMNT and MATCHAMNT PrevAmnt
from pyspark.sql.functions import col
clean_indiv_2001_df = df11.withColumn("TotalAmount", col("Amount")+col("MatchAmt")+col('PrevAmt'))
clean_indiv_2001_df.show()

+--------+-----------+---------------+---------------+--------------------+--------------------+-----------+----------+-----+-------+------+--------+-------+---------+-----------+
|Election|CandidateID|  CandidateName|           Date|                Name|     ContributorType|BoroughName|      City|State|ZipCode|Amount|MatchAmt|PrevAmt|PayMethod|TotalAmount|
+--------+-----------+---------------+---------------+--------------------+--------------------+-----------+----------+-----+-------+------+--------+-------+---------+-----------+
|    2001|         AY|Fernando,Ferrer|10/22/2001 0:00|                null| Candidate Committee|       null|      null| null|   null|   500|       0|      0|    Check|      500.0|
|    2001|         AY|Fernando,Ferrer|10/22/2001 0:00|                null| Candidate Committee|       null|      null| null|   null|  1000|       0|      0|    Check|     1000.0|
|    2001|         AY|Fernando,Ferrer| 10/9/2001 0:00|                null| Candidate Committee|    

In [26]:
#Export to Clean CSV
#clean_indiv_2001_df.toPandas().to_csv("Clean_Indivdual_2005.csv", header=True)

## **2001 Committee Contributions**

In [27]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://databootcamp-final-05.s3.amazonaws.com/Resources/Committee+Contributions+2001+(X).csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("Committee+Contributions+2001+(X).csv"), sep=",", header=True)
df.show()

+--------+--------+-------+--------+----------------+---------+------+--------+------+----------+--------+----------+----------+--------------------+------+-----+-------+---------+---------+--------+-----+-----+----------+--------------------+--------+--------------+------------+--------+-----+---------+--------+----------+--------+------------------+--------+--------+--------+---------------+-----+------+--------------------+----------+-------------------+----------+--------+--------------+---------+--------+---------+------+-------+----------+
|ELECTION|OFFICECD|RECIPID|CANCLASS|       RECIPNAME|COMMITTEE|FILING|SCHEDULE|PAGENO|SEQUENCENO|   REFNO|      DATE|REFUNDDATE|                NAME|C_CODE|STRNO|STRNAME|APARTMENT|BOROUGHCD|    CITY|STATE|  ZIP|OCCUPATION|             EMPNAME|EMPSTRNO|    EMPSTRNAME|     EMPCITY|EMPSTATE| AMNT|MATCHAMNT|PREVAMNT|PAY_METHOD|INTERMNO|        INTERMNAME|INTSTRNO|INTSTRNM|INTAPTNO|        INTCITY|INTST|INTZIP|          INTEMPNAME|INTEMPSTNO|       

In [28]:
# Remove multiple columns
list = ['COMMITTEE', 'FILING', 'SCHEDULE', 'PAGENO', 'SEQUENCENO','REFNO', 'STRNO', 'STRNAME', 'APARTMENT', 'OCCUPATION', 'EMPNAME', 'EMPSTRNO', 'EMPSTRNAME',
                'EMPCITY','REFUNDDATE','INTERMNO', 'INTERMNAME', 'INTSTRNO', 'INTSTRNM', 'INTSTRNM', 'INTAPTNO',
                'INTCITY', 'INTST','INTZIP', 'INTEMPNAME', 'INTEMPSTNO', 'INTEMPSTNM', 'INTEMPCITY', 'INTEMPST', 'INTOCCUPA' ,'PURPOSECD', 'EXEMPTCD','ADJTYPECD', 'RR_IND', 'SEG_IND','INT_C_CODE'] 
df = df.drop(*list)
df.show()

+--------+--------+-------+--------+----------------+----------+--------------------+------+---------+--------+-----+-----+--------+-----+---------+--------+----------+
|ELECTION|OFFICECD|RECIPID|CANCLASS|       RECIPNAME|      DATE|                NAME|C_CODE|BOROUGHCD|    CITY|STATE|  ZIP|EMPSTATE| AMNT|MATCHAMNT|PREVAMNT|PAY_METHOD|
+--------+--------+-------+--------+----------------+----------+--------------------+------+---------+--------+-----+-----+--------+-----+---------+--------+----------+
|    2001|       1|     AY|       P|Ferrer, Fernando| 10/9/2001|                null| PCOMC|     null|    null| null| null|      NY| 1000|        0|       0|         2|
|    2001|       1|     AY|       P|Ferrer, Fernando|10/22/2001|                null| PCOMC|     null|    null| null| null|      NY|  500|        0|       0|         2|
|    2001|       1|     AY|       P|Ferrer, Fernando|10/22/2001|                null| PCOMC|     null|    null| null| null|      NY| 1000|        0|       

In [29]:
# check data types
df.dtypes

[('ELECTION', 'string'),
 ('OFFICECD', 'string'),
 ('RECIPID', 'string'),
 ('CANCLASS', 'string'),
 ('RECIPNAME', 'string'),
 ('DATE', 'string'),
 ('NAME', 'string'),
 ('C_CODE', 'string'),
 ('BOROUGHCD', 'string'),
 ('CITY', 'string'),
 ('STATE', 'string'),
 ('ZIP', 'string'),
 ('EMPSTATE', 'string'),
 ('AMNT', 'string'),
 ('MATCHAMNT', 'string'),
 ('PREVAMNT', 'string'),
 ('PAY_METHOD', 'string')]

In [30]:
#Change column names
df1 = df \
.withColumnRenamed("ELECTION", "Election") \
.withColumnRenamed("RECIPID", "CandidateID") \
.withColumnRenamed("RECIPNAME", "CandidateName") \
.withColumnRenamed("DATE", "Date") \
.withColumnRenamed("NAME", "Name") \
.withColumnRenamed("C_CODE", "ContributorType") \
.withColumnRenamed("BOROUGHCD", "BoroughName") \
.withColumnRenamed("CITY", "City") \
.withColumnRenamed("STATE", "State") \
.withColumnRenamed("ZIP", "ZipCode") \
.withColumnRenamed("EMPSTATE", "ContributionState") \
.withColumnRenamed("AMNT", "Amount") \
.withColumnRenamed("MATCHAMNT", "MatchAmt") \
.withColumnRenamed("PREVAMNT", "PrevAmt") \
.withColumnRenamed("PAY_METHOD", "PayMethod") 

df1.printSchema()

root
 |-- Election: string (nullable = true)
 |-- OFFICECD: string (nullable = true)
 |-- CandidateID: string (nullable = true)
 |-- CANCLASS: string (nullable = true)
 |-- CandidateName: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- ContributorType: string (nullable = true)
 |-- BoroughName: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- ZipCode: string (nullable = true)
 |-- ContributionState: string (nullable = true)
 |-- Amount: string (nullable = true)
 |-- MatchAmt: string (nullable = true)
 |-- PrevAmt: string (nullable = true)
 |-- PayMethod: string (nullable = true)



In [31]:
from datetime import datetime
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType

In [32]:
# using lambda function to convert date col to datetype
# changing datatypes of columns
func =  udf (lambda x: datetime.strptime(x, '%m/%d/%Y'), DateType())

In [33]:
#df2 = df1.withColumn("Election",df1["Election"].cast("Integer")) \
  #.withColumn("OFFICECD", df1["OFFICECD"].cast("Integer")) \
  #.withColumn("ZipCode", df1["ZipCode"].cast("Integer")) \
  #.withColumn("Amount", df1["Amount"].cast("Float")) \
  #.withColumn("MatchAmt", df1["MatchAmt"].cast("Float")) \
  #.withColumn("PrevAmt", df1["PrevAmt"].cast("Float")) \
  #.withColumn('Date', func(col('Date')))
#df2.printSchema()

#Keep Candidate ID as string
#.withColumn("CandidateID", df1["CandidateID"].cast("Integer")) \

In [34]:
df1.show()

+--------+--------+-----------+--------+----------------+----------+--------------------+---------------+-----------+--------+-----+-------+-----------------+------+--------+-------+---------+
|Election|OFFICECD|CandidateID|CANCLASS|   CandidateName|      Date|                Name|ContributorType|BoroughName|    City|State|ZipCode|ContributionState|Amount|MatchAmt|PrevAmt|PayMethod|
+--------+--------+-----------+--------+----------------+----------+--------------------+---------------+-----------+--------+-----+-------+-----------------+------+--------+-------+---------+
|    2001|       1|         AY|       P|Ferrer, Fernando| 10/9/2001|                null|          PCOMC|       null|    null| null|   null|               NY|  1000|       0|      0|        2|
|    2001|       1|         AY|       P|Ferrer, Fernando|10/22/2001|                null|          PCOMC|       null|    null| null|   null|               NY|   500|       0|      0|        2|
|    2001|       1|         AY|    

In [35]:
#Change vaule name in ContributerType
from pyspark.sql.functions import regexp_replace
df3 = df1.withColumn('ContributorType', regexp_replace('ContributorType', 'CAN', 'Candidate')) \
  .withColumn('ContributorType', regexp_replace('ContributorType', 'CORP', 'Corporation')) \
  .withColumn('ContributorType', regexp_replace('ContributorType', 'EMPO', 'Labor Union')) \
  .withColumn('ContributorType', regexp_replace('ContributorType', 'FAM', 'Candidate Family')) \
  .withColumn('ContributorType', regexp_replace('ContributorType', 'IND', 'Individual')) \
  .withColumn('ContributorType', regexp_replace('ContributorType', 'PCOMZ', 'Party Committee')) \
  .withColumn('ContributorType', regexp_replace('ContributorType', 'ORG', 'Orgainization')) \
  .withColumn('ContributorType', regexp_replace('ContributorType', 'LLC', 'Limited Liability Company')) \
  .withColumn('ContributorType', regexp_replace('ContributorType', 'PCOMP', 'Political Action Committee')) \
  .withColumn('ContributorType', regexp_replace('ContributorType', 'SPO', 'Spouse')) \
  .withColumn('ContributorType', regexp_replace('ContributorType', 'OTHR', 'Other')) \
  .withColumn('ContributorType', regexp_replace('ContributorType', 'PCOMC', 'Candidate Committee')) \
  .withColumn('ContributorType', regexp_replace('ContributorType', 'PART', 'Individual')) \


In [36]:
#Change Payment Method Name
df4=df3.withColumn('PayMethod', regexp_replace('PayMethod','0','Unknown')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','1','Cash')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','2','Check')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','3','Other')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','4','Credit Card')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','5','Money Order')) \

In [37]:
#Change Borough name
from pyspark.sql.functions import regexp_replace
df5 = df4.withColumn('BoroughName', regexp_replace('BoroughName', 'K', 'Brooklyn')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'M', 'Manhattan')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'Q', 'Queens')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'S', 'Staten Island')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'X', 'Bronx')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'Z', 'Other')) \


In [38]:
#Call only Mayor (1) and particaptes (P) within Dataframe
df6 = df5.filter((df5.OFFICECD=="1") & (df5.CANCLASS=="P"))
df6.show()

+--------+--------+-----------+--------+----------------+----------+--------------------+--------------------+-----------+--------+-----+-------+-----------------+------+--------+-------+---------+
|Election|OFFICECD|CandidateID|CANCLASS|   CandidateName|      Date|                Name|     ContributorType|BoroughName|    City|State|ZipCode|ContributionState|Amount|MatchAmt|PrevAmt|PayMethod|
+--------+--------+-----------+--------+----------------+----------+--------------------+--------------------+-----------+--------+-----+-------+-----------------+------+--------+-------+---------+
|    2001|       1|         AY|       P|Ferrer, Fernando| 10/9/2001|                null| Candidate Committee|       null|    null| null|   null|               NY|  1000|       0|      0|    Check|
|    2001|       1|         AY|       P|Ferrer, Fernando|10/22/2001|                null| Candidate Committee|       null|    null| null|   null|               NY|   500|       0|      0|    Check|
|    2001|

In [39]:
#Check the above dataframe to make sure only unique values between the OFFICECD is 1 
df6.select('OFFICECD').distinct().show()

+--------+
|OFFICECD|
+--------+
|       1|
+--------+



In [40]:
#Drop OFFICECD and CANCLASS
list2 = ['OFFICECD', 'CANCLASS'] 
df7 = df6.drop(*list2)
df7.show()

+--------+-----------+----------------+----------+--------------------+--------------------+-----------+--------+-----+-------+-----------------+------+--------+-------+---------+
|Election|CandidateID|   CandidateName|      Date|                Name|     ContributorType|BoroughName|    City|State|ZipCode|ContributionState|Amount|MatchAmt|PrevAmt|PayMethod|
+--------+-----------+----------------+----------+--------------------+--------------------+-----------+--------+-----+-------+-----------------+------+--------+-------+---------+
|    2001|         AY|Ferrer, Fernando| 10/9/2001|                null| Candidate Committee|       null|    null| null|   null|               NY|  1000|       0|      0|    Check|
|    2001|         AY|Ferrer, Fernando|10/22/2001|                null| Candidate Committee|       null|    null| null|   null|               NY|   500|       0|      0|    Check|
|    2001|         AY|Ferrer, Fernando|10/22/2001|                null| Candidate Committee|       n

In [41]:
#Filter Dataframe removing the null CandidateID values
df8 = df7.filter(df7.CandidateID.isNotNull())

In [42]:
# ADD AMNT and MATCHAMNT PrevAmnt
from pyspark.sql.functions import col
clean_comm_2001_df = df8.withColumn("TotalAmount", col("Amount")+col("MatchAmt")+col('PrevAmt'))
clean_comm_2001_df.show()

+--------+-----------+----------------+----------+--------------------+--------------------+-----------+--------+-----+-------+-----------------+------+--------+-------+---------+-----------+
|Election|CandidateID|   CandidateName|      Date|                Name|     ContributorType|BoroughName|    City|State|ZipCode|ContributionState|Amount|MatchAmt|PrevAmt|PayMethod|TotalAmount|
+--------+-----------+----------------+----------+--------------------+--------------------+-----------+--------+-----+-------+-----------------+------+--------+-------+---------+-----------+
|    2001|         AY|Ferrer, Fernando| 10/9/2001|                null| Candidate Committee|       null|    null| null|   null|               NY|  1000|       0|      0|    Check|     1000.0|
|    2001|         AY|Ferrer, Fernando|10/22/2001|                null| Candidate Committee|       null|    null| null|   null|               NY|   500|       0|      0|    Check|      500.0|
|    2001|         AY|Ferrer, Fernando|1

In [43]:
# import modules
from pyspark.sql import SparkSession
import functools
  
# explicit function
def unionAll(dfs):
    return functools.reduce(lambda clean_indiv_2001_df, clean_comm_2001_df: clean_indiv_2001_df.union(clean_comm_2001_df.select(clean_indiv_2001_df.columns)), dfs)

unioned_df = unionAll([clean_indiv_2001_df, clean_comm_2001_df])
unioned_df.show()

+--------+-----------+---------------+---------------+--------------------+--------------------+-----------+----------+-----+-------+------+--------+-------+---------+-----------+
|Election|CandidateID|  CandidateName|           Date|                Name|     ContributorType|BoroughName|      City|State|ZipCode|Amount|MatchAmt|PrevAmt|PayMethod|TotalAmount|
+--------+-----------+---------------+---------------+--------------------+--------------------+-----------+----------+-----+-------+------+--------+-------+---------+-----------+
|    2001|         AY|Fernando,Ferrer|10/22/2001 0:00|                null| Candidate Committee|       null|      null| null|   null|   500|       0|      0|    Check|      500.0|
|    2001|         AY|Fernando,Ferrer|10/22/2001 0:00|                null| Candidate Committee|       null|      null| null|   null|  1000|       0|      0|    Check|     1000.0|
|    2001|         AY|Fernando,Ferrer| 10/9/2001 0:00|                null| Candidate Committee|    

In [44]:
#Export to new CSV
unioned_df.toPandas().to_csv("Contributions_2001.csv", header=True)

In [45]:
#Export to new CSV
#clean_comm_2005_df.toPandas().to_csv("Clean_Committee_2005.csv", header=True)

In [46]:
# Store environmental variable
from getpass import getpass
#Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-final.cjdbapst3wjf.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres",
          "password": "******",
          "driver":"org.postgresql.Driver"}

In [47]:
# Write DataFrame to active_user table in RDS
#clean_indiv_2001_df.write.jdbc(url=jdbc_url, table='individual_2001', mode=mode, properties=config)

In [48]:
# Write DataFrame to active_user table in RDS
#clean_comm_2001_df.write.jdbc(url=jdbc_url, table='committee_2001', mode=mode, properties=config)